In [1]:
import numpy as np
import pickle
import os

seed = 2023

In [2]:
import torch

# set device
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
import transformers

# set to only report critical errors to avoid excessing logging
transformers.utils.logging.set_verbosity(50)

In [4]:
from nlpsig_networks.scripts.fine_tune_bert_classification import (
    fine_tune_transformer_average_seed,
)

In [5]:
output_dir = "client_talk_type_output"
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

## AnnoMI

In [6]:
%run ../load_anno_mi.py

In [7]:
anno_mi.head()

,mi_quality,transcript_id,topic,utterance_id,interlocutor,timestamp,utterance_text,annotator_id,therapist_input_exists,therapist_input_subtype,reflection_exists,reflection_subtype,question_exists,question_subtype,main_therapist_behaviour,client_talk_type,datetime,speaker
0,high,0,reducing alcohol consumption,0,therapist,00:00:13,Thanks for filling it out. We give this form t...,3,False,NaN,False,NaN,True,open,question,NaN,2023-11-09 00:00:13,-1
1,high,0,reducing alcohol consumption,1,client,00:00:24,Sure.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-09 00:00:24,1
2,high,0,reducing alcohol consumption,2,therapist,00:00:25,"So, let's see. It looks that you put-- You dri...",3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-09 00:00:25,-1
3,high,0,reducing alcohol consumption,3,client,00:00:34,Mm-hmm.,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,neutral,2023-11-09 00:00:34,1
4,high,0,reducing alcohol consumption,4,therapist,00:00:34,-and you usually have three to four drinks whe...,3,True,information,False,NaN,False,NaN,therapist_input,NaN,2023-11-09 00:00:34,-1


In [8]:
with open("../anno_mi_sbert.pkl", "rb") as f:
    sbert_embeddings = pickle.load(f)

sbert_embeddings.shape

(9699, 384)

## Baseline: Fine-tune BERT for classification

In [9]:
num_epochs = 5
learning_rates = [5e-5, 1e-5, 1e-6]
seeds = [1, 12, 123]
validation_metric = "f1"

In [10]:
label_to_id_client

{'neutral': 0, 'change': 1, 'sustain': 2}

In [11]:
id_to_label_client

{0: 'neutral', 1: 'change', 2: 'sustain'}

In [12]:
kwargs = {
    "num_epochs": num_epochs,
    "pretrained_model_name": "bert-base-uncased",
    "df": anno_mi,
    "feature_name": "utterance_text",
    "label_column": "client_talk_type",
    "label_to_id": label_to_id_client,
    "id_to_label": id_to_label_client,
    "output_dim": output_dim_client,
    "learning_rates": learning_rates,
    "seeds": seeds,
    "device": device,
    "batch_size": 8,
    "path_indices": client_index,
    "split_ids": client_transcript_id,
    "k_fold": True,
    "validation_metric": validation_metric,
    "verbose": False,
}

## Focal Loss

In [13]:
loss = "focal"
gamma = 2

In [14]:
bert_classifier = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_focal.csv",
    **kwargs,
)

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.675592,0.659985,0.416835
2,0.669000,0.635145,0.616808,0.522742
3,0.669000,1.108824,0.639938,0.487981
4,0.478000,1.217864,0.663840,0.530595
5,0.186100,1.401290,0.667695,0.528114


...

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.635850,0.682967,0.503953
2,0.640800,0.625781,0.658517,0.534538
3,0.640800,0.692839,0.668297,0.531480
4,0.465600,0.718743,0.680522,0.543440
5,0.340000,0.759207,0.678077,0.542360


saving the results dataframe to CSV in client_talk_type_output/bert_classifier_focal.csv


In [15]:
bert_classifier

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_accuracy,valid_f1,valid_f1_scores,...,valid_precision_scores,valid_recall,valid_recall_scores,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.644061,0.516876,"[0.7660937955141275, 0.4453816954871362, 0.339...",0.513228,"[0.7717136150234741, 0.455565142364107, 0.3124...",0.522369,"[0.7605552342394447, 0.43564356435643564, 0.37...",0.666827,0.548846,"[0.7863608183508989, 0.4844290657439447, 0.375...",...,"[0.774169921875, 0.5147058823529411, 0.3652849...",0.547763,"[0.798941798941799, 0.45751633986928103, 0.386...",0.000001,1,focal,2,True,5,8
0,0.642720,0.515995,"[0.764490095377843, 0.4560394412489729, 0.3274...",0.511459,"[0.7759904676794758, 0.4541734860883797, 0.304...",0.521931,"[0.7533256217466744, 0.45792079207920794, 0.35...",0.667148,0.549964,"[0.78615326727706, 0.5013315579227696, 0.36240...",...,"[0.7827172827172827, 0.510854816824966, 0.3573...",0.549801,"[0.7896195515243134, 0.492156862745098, 0.3676...",0.000001,12,focal,2,True,5,8
0,0.650192,0.523013,"[0.7708092485549133, 0.4469863616366036, 0.351...",0.521120,"[0.770363951473137, 0.47879359095193214, 0.314...",0.529526,"[0.771255060728745, 0.41914191419141916, 0.398...",0.674534,0.555698,"[0.7914004914004915, 0.4916399857701885, 0.384...",...,"[0.7722368736514026, 0.5394223263075723, 0.372...",0.553202,"[0.8115394305870496, 0.4516339869281046, 0.396...",0.000001,123,focal,2,True,5,8


In [16]:
bert_classifier["f1"].mean()

0.5186279622305993

In [17]:
bert_classifier["precision"].mean()

0.5152689378046481

In [18]:
bert_classifier["recall"].mean()

0.5246087278864766

In [19]:
np.stack(bert_classifier["f1_scores"]).mean(axis=0)

array([0.76713105, 0.44946917, 0.33928367])

In [20]:
np.stack(bert_classifier["precision_scores"]).mean(axis=0)

array([0.77268934, 0.46284407, 0.3102734 ])

In [21]:
np.stack(bert_classifier["recall_scores"]).mean(axis=0)

array([0.76171197, 0.43756876, 0.37454545])

## Using Cross-Entropy loss

In [22]:
loss = "cross_entropy"
gamma = None

In [23]:
bert_classifier_ce = fine_tune_transformer_average_seed(
    loss=loss,
    gamma=gamma,
    results_output=f"{output_dir}/bert_classifier_ce.csv",
    **kwargs,
)

  0%|          | 0/3 [00:00<?, ?it/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.785765,0.672321,0.432937
2,0.775400,0.776554,0.683115,0.534713
3,0.775400,1.093161,0.667695,0.505132
4,0.489000,1.485090,0.673863,0.551776
5,0.232900,1.669982,0.670008,0.553317


...

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

Map:   0%|          | 0/4817 [00:00<?, ? examples/s]

/bask/projects/v/vjgo8416-rough-paths/envs/nlpsig-networks/lib/python3.8/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Validation accuracy,Validation f1
1,No log,0.773512,0.686227,0.395474
2,0.766300,0.751092,0.679707,0.432649
3,0.766300,0.764598,0.686227,0.495828
4,0.595600,0.784316,0.692747,0.524185
5,0.481500,0.810852,0.687857,0.518335


saving the results dataframe to CSV in client_talk_type_output/bert_classifier_ce.csv


In [24]:
bert_classifier_ce

,accuracy,f1,f1_scores,precision,precision_scores,recall,recall_scores,valid_accuracy,valid_f1,valid_f1_scores,...,valid_precision_scores,valid_recall,valid_recall_scores,learning_rate,seed,loss_function,gamma,k_fold,n_splits,batch_size
0,0.660153,0.492106,"[0.783697315342885, 0.4376330353341848, 0.2549...",0.511266,"[0.7550254623425355, 0.4520668425681618, 0.326...",0.482605,"[0.8146327356853673, 0.4240924092409241, 0.209...",0.677103,0.518548,"[0.7980167630740174, 0.4768632991875662, 0.280...",...,"[0.7507774322523323, 0.5188316679477325, 0.381...",0.505000,"[0.8515998992189469, 0.4411764705882353, 0.222...",0.000001,1,cross_entropy,None,True,5,8
0,0.656513,0.503736,"[0.7794784580498867, 0.45109612141652616, 0.28...",0.510960,"[0.7643135075041689, 0.46120689655172414, 0.30...",0.498286,"[0.7952573742047426, 0.4414191419141914, 0.258...",0.677425,0.538457,"[0.7971452764001452, 0.49140893470790376, 0.32...",...,"[0.7666356444858073, 0.5181159420289855, 0.38]",0.528066,"[0.8301839254220207, 0.4673202614379085, 0.286...",0.000001,12,cross_entropy,None,True,5,8
0,0.668582,0.507366,"[0.7892995035852179, 0.4374152733845459, 0.295...",0.525563,"[0.7543489720611491, 0.4835164835164835, 0.338...",0.496268,"[0.8276460381723539, 0.39933993399339934, 0.26...",0.684329,0.533591,"[0.8031939877876938, 0.4745762711864407, 0.323...",...,"[0.7521442709478777, 0.543918918918919, 0.3983...",0.518066,"[0.8616780045351474, 0.42091503267973857, 0.27...",0.000001,123,cross_entropy,None,True,5,8


In [25]:
bert_classifier_ce["f1"].mean()

0.5010695163433949

In [26]:
bert_classifier_ce["precision"].mean()

0.5159295051966489

In [27]:
bert_classifier_ce["recall"].mean()

0.4923865047002098

In [28]:
np.stack(bert_classifier_ce["f1_scores"]).mean(axis=0)

array([0.78415843, 0.44204814, 0.27700198])

In [29]:
np.stack(bert_classifier_ce["precision_scores"]).mean(axis=0)

array([0.75789598, 0.46559674, 0.32429579])

In [30]:
np.stack(bert_classifier_ce["recall_scores"]).mean(axis=0)

array([0.81251205, 0.42161716, 0.2430303 ])